In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
video_files = glob.glob('video_footages/*')
video_files

['video_footages\\Raw Video_ Pittsburg Neighborhood Drive-By Shootings.mp4',
 'video_footages\\Shooting captured by surveillance camera in Parma.mp4',
 'video_footages\\Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4',
 'video_footages\\snow dispute.mp4',
 'video_footages\\Surveillance camera captures tanks entering Ukraine via Belarus border.mp4',
 'video_footages\\trail cam video creatures of the night.mp4',
 'video_footages\\Unbelievably busy bicycle crossing in Amsterdam.mp4',
 'video_footages\\vecteezy_people-crossing-the-road-on-zebra-tallin_28257759.mp4',
 'video_footages\\차흐르듯 (1).mp4',
 'video_footages\\차흐르듯.mp4']

# Sparse Optical Flow

In [6]:
video_num = 9

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
 qualityLevel = 0.3,
 minDistance = 7,
 blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize = (15, 15),
 maxLevel = 2,
 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

color = np.random.randint(0, 255, (100, 3))

cap = cv2.VideoCapture(video_files[video_num])

ret, old_frame = cap.read()
new_h = 300
new_w = 600
old_frame = cv2.resize(old_frame, (new_w, new_h))
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
prev = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

mask = np.zeros_like(old_frame)

count = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    count += 1

    frame = cv2.resize(frame, (new_w, new_h))
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    
    # calculate optical flow
    next, status, error = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, prev, None, **lk_params)
    
    good_old = prev[status == 1]
    good_new = next[status == 1]

    img = frame.copy()
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        img = cv2.circle(img, (int(a), int(b)), 3, color[i].tolist(), -1)

    img = cv2.add(img, mask)
    cv2.imshow('optical flow', img)
    cv2.imshow('original', frame)
    k = cv2.waitKey(30) & 0xff
    if k == ord('q'):
        break
    elif k == ord('s'):
        cv2.imwrite(f'saved images for essay/sparse optical flow/{video_num}_{count}_original.jpg', frame)
        cv2.imwrite(f'saved images for essay/sparse optical flow/{video_num}_{count}_optical_flow.jpg', img)
    
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    prev = good_new.reshape(-1, 1, 2)
    
cv2.destroyAllWindows()

# Dense Optical Flow

In [3]:
def get_motion_mask(flow_mag, motion_thresh=1, kernel=np.ones((7,7))):

    motion_mask = np.uint8(flow_mag > motion_thresh)*255

    motion_mask = cv2.erode(motion_mask, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel, iterations=1)
    
    return motion_mask

In [5]:
video_num = 5
count = 0
cap = cv2.VideoCapture(video_files[video_num])

ret, frame1 = cap.read()
new_h = 300
new_w = 600
kernel = np.ones((5, 5), np.uint8)
frame1 = cv2.resize(frame1, (new_w, new_h))

prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(cap.isOpened()):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break
    count += 1
    frame2 = cv2.resize(frame2, (new_w, new_h))
    next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    #next = cv2.GaussianBlur(next, (21, 21), 0)
    #next = cv2.medianBlur(next, 21)
    flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    #fx = cv2.threshold(bgr, 5, 255, cv2.THRESH_BINARY)[1]
    #fx = cv2.cvtColor(fx, cv2.COLOR_BGR2GRAY)
    #fx = cv2.threshold(fx, 0, 255, cv2.THRESH_BINARY)[1]
    #fx = cv2.erode(fx, kernel, iterations=2)
    fx = get_motion_mask(mag)
    contours, _ = cv2.findContours(fx, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_L1)
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        area = w*h
        if area>200:
            cv2.rectangle(frame2, (x, y), (x+w, y+h), (0, 255, 0), thickness = 2)
    cv2.imshow('Original', frame2)
    cv2.imshow('Optical flow', bgr)
    cv2.imshow('mask', fx)
    k = cv2.waitKey(1) & 0xff
    if k == ord('q'):
        break
    elif k == ord('s'):
        cv2.imwrite(f'saved images for essay/dense optical flow/{video_num}_{count}_original.jpg', frame2)
        cv2.imwrite(f'saved images for essay/dense optical flow/{video_num}_{count}_optical flow.jpg', bgr)
        cv2.imwrite(f'saved images for essay/dense optical flow/{video_num}_{count}_mask.jpg', fx)
        
    #elif k == ord('s'):
    #    cv2.imwrite('opticalfb.png', frame2)
    #    cv2.imwrite('opticalhsv.png', bgr)
    prvs = next

cv2.destroyAllWindows()